<p align = "center">
    UNIVERSIDADE FEDERAL DE CAMPINA GRANDE <br>
    CENTRO DE ENGENHARIA ELÉTRICA E INFORMÁTICA <br>
    UNIDADE ACADÊMICA DE ENGENHARIA ELÉTRICA <br>
    DISCIPLINA: CIRCUITOS ELÉTRICOS 2 <br>
</p>

<h2 align = "center"> Modelagem de Sistema Elétrico de Potência</h2>



<p> Este projeto tem como objetivo realizar o projeto e dimensionamento de um Sistema Elétrico de Potência (SEP) utilzando a teoria de Quadripolos. </p>
<p> Projetistas: </p>
<ul> 
    <li> André Cadena de Oliveira
    <li> Fabio Marçal
    <li> Gabriel Nazário
    <li> Lorenzo Carrera de Oliveira
    <li> Mateus Pincho de Oliveira
</ul> 


---
 
 Segundo a teoria de quadripolos, podemos modelar os componentes do sistema elétrico de potência da seguinte forma: 

### Linha de transmissão

<p align = "center">
    <img src = '../docs/linha.jpeg'>
</p>

Linhas de transmissão possuem as suas características distríbuidas por toda linha, sendo a impedância $Z = R + jX_L$ e sua admitância $Y = jX_C$. Para linhas de transmissão entre 80 a 250 quilômetros, podemos concentrar os seus parâmetros no modelo a seguir. A tensão e corrente no começo e final da linha de transmissão podem ser determinadas como:

$$ \begin{bmatrix}
V_1\\ 
I_1
\end{bmatrix} = \begin{bmatrix}
1 + \frac{ZY}{2} & Z\\ 
\frac{4Y^2+2Y}{4} & 1 + \frac{ZY}{2}
\end{bmatrix} \cdot \begin{bmatrix}
V_2\\ 
-I_2
\end{bmatrix} $$

---

### Transformador

<p align = "center">
    <img src = '../docs/transformador.jpeg'>
</p>

Um transformador de um sistema elétrico de potência pode ser modelador como mostrado na figura anterior. A relação de tensão e corrente nos seus terminais pode ser escrita como: 

$$
\begin{bmatrix}
V_1\\ 
I_1
\end{bmatrix} = \begin{bmatrix}
\frac{1}{n}(1+YZ_1) &n(Z_1+Z_2+YZ_1Z_2) \\ 
\frac{1}{n}Y & n(1+YZ_2)
\end{bmatrix}\cdot \begin{bmatrix}
V_2\\ 
-I_2
\end{bmatrix}
$$

Sendo: 

$$Z_1 = R_1+jX_{L_1}$$
$$Z_2 = R_2+jX_{L_2}$$

$$Y = \frac{R_m + jX_m}{R_m \cdot jX_m}$$

---

### Carga em série

A tensão e corrente nos terminais de uma carga colocada em série com o circuito é dada por:

$$\begin{bmatrix}
V_1\\ 
I_1
\end{bmatrix} = \begin{bmatrix}
1 &Z \\ 
0 & 1
\end{bmatrix}\cdot \begin{bmatrix}
V_2\\ 
-I_2
\end{bmatrix} $$

---

### Carga em paralelo

A tensão e corrente nos terminais de uma carga colocada em paralelo com o circuito é dada por:

$$ \begin{bmatrix}
V_1\\ 
I_1
\end{bmatrix} = \begin{bmatrix}
1 &0 \\ 
Y & 1
\end{bmatrix}\cdot \begin{bmatrix}
V_2\\ 
-I_2
\end{bmatrix}$$

---

## Questão 01

Considerando o sistema elétrico abaixo operando a $60 \text{ Hz}$:

<p align = "center">
    <img src = '../docs/sistema-eletrico.jpeg'>
</p>

Modele este sistema criando as matrizes de trasnferência $[T]$ para cada elemento a partir dos dados de entrada do modelo. Caso dois elementos estejam conectados em série, deve-se realizar a conexão em cascata, se não, deve ser feita a conexão em paralelo.

Para realizar a modelagem, considere os seguintes parâmetros: 
- Impedância em série de Thévenin: $R_f = 4\Omega$ e $X_f = 0.38\Omega$
- Parâmetros distribuídos da linha de transmissão: $R = 0.17 \Omega\text{/km}$, $L = 2.18 \text{ mH/km}$
- Considere que ${LT}_1, {LT}_2$ e ${LT}_3$ possuem $80 \text{ km}$ e que ${LT}_4$ possui $120 \text{ km}$


In [ ]:
#Funções para criar a matriz de transferência